In [65]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [51]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

In [38]:
train, test = tf.keras.datasets.cifar10.load_data()

In [39]:
x_train = train[0]
y_train = train[1]
x_test = test[0]
y_test = test[1]

In [77]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
        
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [53]:
# Create the Estimator
cifar10_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./cifar10_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cifar10_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000045BD41D358>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [54]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

In [82]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train.astype(np.float32)},
    y=y_train.astype(np.int32),
    batch_size=100,
    num_epochs=None,
    shuffle=True)
cifar10_classifier.train(
    input_fn=train_input_fn,
    steps=10000,
    hooks=[])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model\model.ckpt-1100
INFO:tensorflow:Saving checkpoints for 1101 into ./cifar10_convnet_model\model.ckpt.
INFO:tensorflow:loss = 2.17419, step = 1101
INFO:tensorflow:global_step/sec: 1.70568
INFO:tensorflow:loss = 2.18893, step = 1201 (58.632 sec)
INFO:tensorflow:global_step/sec: 2.0484
INFO:tensorflow:loss = 2.26661, step = 1301 (48.818 sec)
INFO:tensorflow:global_step/sec: 2.21857
INFO:tensorflow:loss = 2.22778, step = 1401 (45.084 sec)
INFO:tensorflow:global_step/sec: 2.24358
INFO:tensorflow:loss = 2.19275, step = 1501 (44.562 sec)
INFO:tensorflow:global_step/sec: 2.2378
INFO:tensorflow:loss = 2.23803, step = 1601 (44.686 sec)
INFO:tensorflow:global_step/sec: 2.2382
INFO:tensorflow:loss = 2.19886, step = 1701 (44.749 sec)
INFO:tensorflow:global_step/sec: 2.2313
INFO:tensorflow:loss = 2.11839, step = 1801 (44.747 sec)
INFO:tensorflow:global_step/sec: 2.22782
INFO:tensorflow:loss =

INFO:tensorflow:Saving checkpoints for 8724 into ./cifar10_convnet_model\model.ckpt.
INFO:tensorflow:global_step/sec: 1.64959
INFO:tensorflow:loss = 1.52704, step = 8801 (60.636 sec)
INFO:tensorflow:global_step/sec: 1.57738
INFO:tensorflow:loss = 1.48398, step = 8901 (63.381 sec)
INFO:tensorflow:global_step/sec: 2.20059
INFO:tensorflow:loss = 1.38296, step = 9001 (45.442 sec)
INFO:tensorflow:global_step/sec: 2.2328
INFO:tensorflow:loss = 1.4031, step = 9101 (44.797 sec)
INFO:tensorflow:global_step/sec: 2.2204
INFO:tensorflow:loss = 1.4414, step = 9201 (45.026 sec)
INFO:tensorflow:global_step/sec: 1.7847
INFO:tensorflow:loss = 1.51063, step = 9301 (56.034 sec)
INFO:tensorflow:global_step/sec: 1.84527
INFO:tensorflow:loss = 1.34826, step = 9401 (54.202 sec)
INFO:tensorflow:global_step/sec: 2.18476
INFO:tensorflow:loss = 1.40324, step = 9501 (45.762 sec)
INFO:tensorflow:global_step/sec: 2.16582
INFO:tensorflow:loss = 1.52861, step = 9601 (46.171 sec)
INFO:tensorflow:global_step/sec: 2.215

In [83]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test.astype(np.float32)},
    y=y_test.astype(np.int32),
    num_epochs=1,
    shuffle=False)
eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-02-25-01:43:02
INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model\model.ckpt-11100
INFO:tensorflow:Finished evaluation at 2018-02-25-01:43:19
INFO:tensorflow:Saving dict for global step 11100: accuracy = 0.5211, global_step = 11100, loss = 1.36193
{'accuracy': 0.52109998, 'loss': 1.3619258, 'global_step': 11100}


In [107]:
# Evaluate the model and print results
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test.astype(np.float32)},
    num_epochs=1,
    shuffle=False)
pred_results = cifar10_classifier.predict(input_fn=pred_input_fn, predict_keys="classes")

<generator object Estimator.predict at 0x00000045B97C2200>


In [108]:
pred_df = pd.DataFrame.from_records(list(pred_results))
pred_df.index.names=['ID']
pred_df.columns = ['CLASSES']
pred_df.to_csv('preds.csv')

INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model\model.ckpt-11100
